In [44]:
import pickle
import pandas as pd
import numpy as np
import os
import re

In [35]:
import sys 
import os
sys.path.append(os.path.abspath("src/Measurement"))
import globals
from src.Measurement.measurement import Measurement
globals.init()
print(globals.API_URL)

import os
import sys
import argparse
from tqdm import tqdm
import pickle
import pandas as pd
import numpy as np


from datasets import concatenate_datasets, load_dataset, DatasetDict, Dataset
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch
import platform
import evaluate
from transformers import AutoTokenizer, BartForConditionalGeneration, GPT2Model, T5ForConditionalGeneration, GPT2LMHeadModel
print(platform.platform())
from transformers import pipeline


from huggingface_hub import login

macOS-14.1-arm64-arm-64bit
https://api-inference.huggingface.co/models/s-nlp/roberta-base-formality-ranker
macOS-14.1-arm64-arm-64bit


In [25]:
def load_pickle(file_path, file_name):
    with open(os.path.join(file_path , file_name), 'rb') as f:
        return pickle.load(f)

In [56]:
rootdir = './test'

subdirs = ['t5/dialogue', 'bart/news']
generations = ['generation0', 'generation1', 'generation2', 'generation']

results = {}

for subdir in subdirs:
    for gen in generations:
        path = os.path.join(rootdir, subdir, gen, 'results')
        results[subdir + '/' + gen] = {
            'config': load_pickle(path, 'config.pkl'),
            'measurements': load_pickle(path, 'measurements.pkl'),
            'test_eval_results': load_pickle(path, 'test_data_eval_results.pkl'),
        }

In [59]:
keys = list(results.keys())
keys

['t5/dialogue/generation0',
 't5/dialogue/generation1',
 't5/dialogue/generation2',
 'bart/news/generation0',
 'bart/news/generation1',
 'bart/news/generation2']

In [61]:
results[keys[2]]['measurements']['samples'][0]

"Amanda: I baked cookies. Do you want some? Jerry: I'll bring you tomorrow:-) Amanda: i've baked cookies ."

In [37]:
def load_huggingface_dataset(data_config):
    data_dict = load_dataset(*data_config)
    dataset = concatenate_datasets([data_dict[k] for k in data_dict.keys()])
    # print(dataset.column_names)
    for col in dataset.column_names:
        if col in globals.new_col_names.keys():
            dataset = dataset.rename_column(col, globals.new_col_names[col])
    #         print(col, globals.new_col_names[col])
    # print(dataset.column_names)
    if args['dataset'] == 'reddit':
        return dataset.select_columns(['document', 'summary'])
    else:
       return dataset.select_columns(['id', 'document', 'summary'])
   

args = {
    'dataset': 'dialogue'
}
dialogue_dataset = load_huggingface_dataset(['samsum'])
dialogue_dataset

Dataset({
    features: ['id', 'document', 'summary'],
    num_rows: 16369
})

In [46]:
x = dialogue_dataset.filter(lambda x: re.search('Amanda', x['document']))

In [49]:
x['document']

["Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)",
 "Tom: hi guys, we're thinking about renting a car for a few days\r\nTom: and have a small trip somewhere over the weekend\r\nTom: wanna participate?\r\nJerry: It's super cheap now, like 30€ for 5 days\r\nAmanda: that's really nothing\r\nAmanda: when do you want to do this?\r\nAlexis: I'd be interested for sure\r\nTom: we thought about the first weekend of January\r\nAlexis: for me it's perfect, no plans here really\r\nAmanda: I'd like to join as well\r\nAmanda: any ideas where we want to go?\r\nTom: we thought about some thermal baths\r\nAmanda: wow, how exciting\r\nAmanda: let's do it!",
 'Amanda: How is the journey going?\r\nRebeca: Amazing, I needed it so much\r\nThomas: Yes, Rebeca is ecstatic about everything here hahahah\r\nRebeca: true, I like food, people, sun, sea, a paradise\r\nAmanda: so envious!\r\nAmanda: Where are you right now?\r\nRebecca: Bangkok \r\nAmanda: one of th